<a href="https://colab.research.google.com/github/rkrissada/google_ml_training/blob/master/b_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [0]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [0]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [0]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [0]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [0]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_service': None, '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_is_chief': True, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_master': '', '_train_distribute': None, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f100fa64d30>, '_tf_random_seed': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 18707.385, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [0]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-29-08:13:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-29-08:13:16
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.33886, global_step = 608, loss = 13003.515
RMSE on dataset = 10.456521987915039


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [0]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.906272], dtype=float32)}
{'predictions': array([10.903724], dtype=float32)}
{'predictions': array([10.904586], dtype=float32)}
{'predictions': array([10.902136], dtype=float32)}
{'predictions': array([10.906011], dtype=float32)}
{'predictions': array([10.905791], dtype=float32)}
{'predictions': array([10.904371], dtype=float32)}
{'predictions': array([10.904402], dtype=float32)}
{'predictions': array([10.906165], dtype=float32)}
{'predictions': array([10.904018], dtype=float32)}
{'predictions': array([10.906274], dtype=float32)}
{'predictions': array([10.90644], dtype=float32)}
{'predictions': array([10.899713], dtype=float32)}
{'predictions': array([10.903634], dtype=float32)}
{'predictions': array([1

{'predictions': array([10.903652], dtype=float32)}
{'predictions': array([10.9014], dtype=float32)}
{'predictions': array([10.90473], dtype=float32)}
{'predictions': array([10.967229], dtype=float32)}
{'predictions': array([10.904212], dtype=float32)}
{'predictions': array([11.0309105], dtype=float32)}
{'predictions': array([10.904509], dtype=float32)}
{'predictions': array([10.906838], dtype=float32)}
{'predictions': array([10.904032], dtype=float32)}
{'predictions': array([10.905737], dtype=float32)}
{'predictions': array([10.90681], dtype=float32)}
{'predictions': array([11.031731], dtype=float32)}
{'predictions': array([10.904466], dtype=float32)}
{'predictions': array([10.904625], dtype=float32)}
{'predictions': array([10.96697], dtype=float32)}
{'predictions': array([11.020133], dtype=float32)}
{'predictions': array([10.906415], dtype=float32)}
{'predictions': array([11.21627], dtype=float32)}
{'predictions': array([10.904439], dtype=float32)}
{'predictions': array([10.905172], d

{'predictions': array([10.903882], dtype=float32)}
{'predictions': array([10.900428], dtype=float32)}
{'predictions': array([11.153315], dtype=float32)}
{'predictions': array([11.154316], dtype=float32)}
{'predictions': array([10.903935], dtype=float32)}
{'predictions': array([10.966172], dtype=float32)}
{'predictions': array([10.901374], dtype=float32)}
{'predictions': array([10.904394], dtype=float32)}
{'predictions': array([10.903643], dtype=float32)}
{'predictions': array([10.905084], dtype=float32)}
{'predictions': array([10.904384], dtype=float32)}
{'predictions': array([10.904206], dtype=float32)}
{'predictions': array([10.907631], dtype=float32)}
{'predictions': array([10.904246], dtype=float32)}
{'predictions': array([10.904452], dtype=float32)}
{'predictions': array([10.903725], dtype=float32)}
{'predictions': array([10.903483], dtype=float32)}
{'predictions': array([11.091857], dtype=float32)}
{'predictions': array([10.966708], dtype=float32)}
{'predictions': array([11.15398

{'predictions': array([11.15304], dtype=float32)}
{'predictions': array([10.9032545], dtype=float32)}
{'predictions': array([10.905964], dtype=float32)}
{'predictions': array([10.967391], dtype=float32)}
{'predictions': array([10.904136], dtype=float32)}
{'predictions': array([10.90445], dtype=float32)}
{'predictions': array([10.903491], dtype=float32)}
{'predictions': array([10.904264], dtype=float32)}
{'predictions': array([10.904016], dtype=float32)}
{'predictions': array([10.906003], dtype=float32)}
{'predictions': array([10.901893], dtype=float32)}
{'predictions': array([10.904105], dtype=float32)}
{'predictions': array([11.15633], dtype=float32)}
{'predictions': array([10.904275], dtype=float32)}
{'predictions': array([10.90417], dtype=float32)}
{'predictions': array([10.90516], dtype=float32)}
{'predictions': array([11.02606], dtype=float32)}
{'predictions': array([10.890842], dtype=float32)}
{'predictions': array([11.217465], dtype=float32)}
{'predictions': array([10.903787], d

{'predictions': array([10.90429], dtype=float32)}
{'predictions': array([10.899904], dtype=float32)}
{'predictions': array([10.903099], dtype=float32)}
{'predictions': array([10.967565], dtype=float32)}
{'predictions': array([10.965762], dtype=float32)}
{'predictions': array([11.152851], dtype=float32)}
{'predictions': array([10.903892], dtype=float32)}
{'predictions': array([10.904226], dtype=float32)}
{'predictions': array([10.90491], dtype=float32)}
{'predictions': array([11.090819], dtype=float32)}
{'predictions': array([10.966472], dtype=float32)}
{'predictions': array([10.90686], dtype=float32)}
{'predictions': array([10.96263], dtype=float32)}
{'predictions': array([10.904376], dtype=float32)}
{'predictions': array([10.904794], dtype=float32)}
{'predictions': array([11.150664], dtype=float32)}
{'predictions': array([11.15362], dtype=float32)}
{'predictions': array([10.904452], dtype=float32)}
{'predictions': array([10.903943], dtype=float32)}
{'predictions': array([11.217258], d

{'predictions': array([10.90446], dtype=float32)}
{'predictions': array([10.904521], dtype=float32)}
{'predictions': array([11.153924], dtype=float32)}
{'predictions': array([11.027696], dtype=float32)}
{'predictions': array([10.90471], dtype=float32)}
{'predictions': array([11.216363], dtype=float32)}
{'predictions': array([10.906527], dtype=float32)}
{'predictions': array([10.903503], dtype=float32)}
{'predictions': array([10.905498], dtype=float32)}
{'predictions': array([10.904685], dtype=float32)}
{'predictions': array([10.904302], dtype=float32)}
{'predictions': array([10.904992], dtype=float32)}
{'predictions': array([10.903126], dtype=float32)}
{'predictions': array([10.906882], dtype=float32)}
{'predictions': array([10.903916], dtype=float32)}
{'predictions': array([10.903528], dtype=float32)}
{'predictions': array([10.903856], dtype=float32)}
{'predictions': array([10.9046135], dtype=float32)}
{'predictions': array([10.904399], dtype=float32)}
{'predictions': array([10.901443

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_service': None, '_session_config': None, '_task_id': 0, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_is_chief': True, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_master': '', '_train_distribute': None, '_evaluation_master': '', '_model_dir': 'taxi_trained', '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0feadd3b00>, '_tf_random_seed': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 27754.195, st

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [0]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [0]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-29-08:13:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-29-08:13:57
INFO:tensorflow:Saving dict for global step 6071: average_loss = 89.0855, global_step = 6071, loss = 11319.038
RMSE on dataset = 9.438511848449707


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License